# Semi-Variance (SV) Risk Measure

In [17]:
import numpy as np
import cvxpy as cp


In [18]:
def calculate_semi_variance(returns):
    # Calculate the mean return for each asset
    mean_returns = np.mean(returns, axis=0)
    
    # Calculate deviations less than the mean (downside)
    downside_deviation = np.minimum(0, returns - mean_returns)
    
    # Calculate the semi-variance
    semi_variance = np.mean(downside_deviation**2, axis=0)
    
    return semi_variance


In [19]:
def optimize_portfolio_semi_variance(returns):
    num_assets = returns.shape[1]
    weights = cp.Variable(num_assets)
    
    # The expected return for the portfolio
    portfolio_return = returns @ weights
    
    # The semi-variance of the portfolio
    mean_portfolio_return = cp.sum(portfolio_return) / len(returns)
    downside_deviation = cp.minimum(0, portfolio_return - mean_portfolio_return)
    portfolio_semi_variance = cp.sum_squares(downside_deviation) / len(returns)
    
    # Objective: minimize the semi-variance
    objective = cp.Minimize(portfolio_semi_variance)
    
    # Constraints: sum of weights = 1, weights must be non-negative (no short-selling)
    constraints = [cp.sum(weights) == 1, weights >= 0]
    
    # Solve the optimization problem
    problem = cp.Problem(objective, constraints)
    problem.solve()
    
    return weights.value


In [20]:
# Example data (randomly generated for demonstration)
np.random.seed(42)
example_returns = np.random.randn(100, 4) * 0.01  # 100 days, 4 assets

# Optimize the portfolio to minimize semi-variance
optimized_weights = optimize_portfolio_semi_variance(example_returns)
print("Optimized Portfolio Weights:", optimized_weights)


Optimized Portfolio Weights: [0.25281384 0.25315089 0.24487327 0.249162  ]


# Mean Absolute Deviation (MAD) Risk Measure

In [21]:
import numpy as np
import cvxpy as cp


In [22]:
def calculate_mean_absolute_deviation(returns):
    # Calculate the mean return for each asset
    mean_returns = np.mean(returns, axis=0)
    
    # Calculate the absolute deviations from the mean
    absolute_deviation = np.abs(returns - mean_returns)
    
    # Calculate the mean absolute deviation
    mad = np.mean(absolute_deviation, axis=0)
    
    return mad


In [23]:
def optimize_portfolio_mad(returns):
    num_assets = returns.shape[1]
    weights = cp.Variable(num_assets)
    
    # The portfolio return
    portfolio_return = returns @ weights
    
    # The mean return of the portfolio
    mean_portfolio_return = cp.sum(portfolio_return) / len(returns)
    
    # The absolute deviations from the mean return of the portfolio
    absolute_deviation = cp.abs(portfolio_return - mean_portfolio_return)
    portfolio_mad = cp.sum(absolute_deviation) / len(returns)
    
    # Objective: minimize the mean absolute deviation
    objective = cp.Minimize(portfolio_mad)
    
    # Constraints: sum of weights = 1, weights must be non-negative (no short-selling)
    constraints = [cp.sum(weights) == 1, weights >= 0]
    
    # Solve the optimization problem
    problem = cp.Problem(objective, constraints)
    problem.solve()
    
    return weights.value


In [24]:
# Example data (randomly generated for demonstration)
np.random.seed(42)
example_returns = np.random.randn(100, 4) * 0.01  # 100 days, 4 assets

# Optimize the portfolio to minimize MAD
optimized_weights = optimize_portfolio_mad(example_returns)
print("Optimized Portfolio Weights:", optimized_weights)


Optimized Portfolio Weights: [0.2843136  0.24261004 0.25018537 0.22289099]


# Semi Absolute Deviation (SAD) Risk Measure

In [25]:
import numpy as np
import cvxpy as cp


In [26]:
def calculate_semi_absolute_deviation(returns, threshold=None):
    if threshold is None:
        threshold = np.mean(returns, axis=0)  # default threshold as the mean return
    
    # Calculate deviations below the threshold
    downside_deviation = np.minimum(0, returns - threshold)
    
    # Calculate the semi absolute deviation
    sad = np.mean(np.abs(downside_deviation), axis=0)
    
    return sad


In [27]:
def optimize_portfolio_sad(returns, threshold=None):
    num_assets = returns.shape[1]
    weights = cp.Variable(num_assets)
    
    # The portfolio return
    portfolio_return = returns @ weights
    
    if threshold is None:
        threshold = cp.sum(portfolio_return) / len(returns)  # Default to mean portfolio return
    
    # Calculate the downside deviations below the threshold
    downside_deviation = cp.minimum(0, portfolio_return - threshold)
    
    # The SAD of the portfolio
    portfolio_sad = cp.sum(cp.abs(downside_deviation)) / len(returns)
    
    # Objective: minimize the semi absolute deviation
    objective = cp.Minimize(portfolio_sad)
    
    # Constraints: sum of weights = 1, weights must be non-negative (no short-selling)
    constraints = [cp.sum(weights) == 1, weights >= 0]
    
    # Solve the optimization problem
    problem = cp.Problem(objective, constraints)
    problem.solve()
    
    return weights.value


In [28]:
# Example data (randomly generated for demonstration)
np.random.seed(42)
example_returns = np.random.randn(100, 4) * 0.01  # 100 days, 4 assets

# Optimize the portfolio to minimize SAD
optimized_weights = optimize_portfolio_sad(example_returns)
print("Optimized Portfolio Weights:", optimized_weights)


Optimized Portfolio Weights: [0.2843136  0.24261004 0.25018537 0.22289099]


 # Optimization and Constraints

In [29]:
import numpy as np
import cvxpy as cp
import sys


In [30]:
pip install ecos

In [31]:
import cvxpy as cp
import numpy as np

def optimize_portfolio_with_constraints(returns, min_assets, max_assets, min_weight, max_weight):
    num_assets = returns.shape[1]
    weights = cp.Variable(num_assets)
    # Introduce binary variables for handling cardinality
    selection = cp.Variable(num_assets, boolean=True)

    # The expected return for the portfolio
    portfolio_return = returns @ weights
    risk = cp.quad_form(weights, np.cov(returns.T))

    # Objective: minimize risk
    objective = cp.Minimize(risk)

    # Constraints
    constraints = [
        cp.sum(weights) == 1,  # Budget constraint
        weights >= min_weight * selection,  # Min weight constraint, only applies if asset is selected
        weights <= max_weight * selection,  # Max weight constraint, only applies if asset is selected
        weights >= 0,  # No short-selling
        cp.sum(selection) >= min_assets,  # Minimum number of assets
        cp.sum(selection) <= max_assets,  # Maximum number of assets
    ]

    # Solve the optimization problem
    problem = cp.Problem(objective, constraints)
    problem.solve(solver=cp.ECOS_BB)  # Use ECOS_BB for mixed-integer conic optimization

    return weights.value

# Example data (randomly generated for demonstration)
np.random.seed(42)
example_returns = np.random.randn(100, 4) * 0.01  # 100 days, 4 assets

# Parameters for constraints
min_assets = 2  # Minimum number of assets in the portfolio
max_assets = 4  # Maximum number of assets in the portfolio
min_weight = 0.1  # Minimum weight of any asset in the portfolio
max_weight = 0.6  # Maximum weight of any asset in the portfolio

# Optimize the portfolio with specified constraints
optimized_weights = optimize_portfolio_with_constraints(
    example_returns, min_assets, max_assets, min_weight, max_weight
)
print("Optimized Portfolio Weights:", optimized_weights)


Optimized Portfolio Weights: [0.31354667 0.23795211 0.22423531 0.2242659 ]


# Stochastic Programming Model

In [32]:
import numpy as np
import cvxpy as cp


In [33]:
# Example market scenarios
np.random.seed(42)
num_assets = 4
num_scenarios = 10
scenario_returns = np.random.randn(num_scenarios, num_assets) * 0.01  # Random returns for illustration
scenario_probabilities = np.full(num_scenarios, 1.0 / num_scenarios)  # Equal probability for each scenario


In [34]:
def minimize_regret(scenario_returns, scenario_probabilities):
    num_scenarios, num_assets = scenario_returns.shape
    weights = cp.Variable(num_assets)
    scenario_contributions = []

    for i in range(num_scenarios):
        # Calculate the portfolio return for scenario i
        portfolio_return = scenario_returns[i] @ weights
        
        # Maximum return possible for this scenario (if we could invest optimally with hindsight)
        best_return = cp.max(scenario_returns[i])
        
        # Contribution to regret from this scenario
        regret = scenario_probabilities[i] * (best_return - portfolio_return)
        scenario_contributions.append(regret)
    
    # Total expected regret
    total_regret = cp.sum(scenario_contributions)
    
    # Objective: Minimize total expected regret
    objective = cp.Minimize(total_regret)
    
    # Constraints: Sum of weights is 1, weights are non-negative
    constraints = [cp.sum(weights) == 1, weights >= 0]
    
    # Solve the problem
    problem = cp.Problem(objective, constraints)
    problem.solve()
    
    return weights.value

# Optimize portfolio to minimize regret
optimized_weights = minimize_regret(scenario_returns, scenario_probabilities)
print("Optimized Portfolio Weights:", optimized_weights)


Optimized Portfolio Weights: [1.09088619e-07 9.31090070e-09 1.25658481e-08 9.99999869e-01]


# Augmented ε-constraint Method

In [35]:
import numpy as np
import cvxpy as cp


In [36]:
np.random.seed(42)
num_assets = 4
returns = np.random.randn(num_assets) * 0.01
covariance_matrix = np.random.randn(num_assets, num_assets) * 0.01
covariance_matrix = covariance_matrix @ covariance_matrix.T  # To ensure it's positive definite


In [37]:
def augmented_epsilon_constraint(returns, covariance, epsilon_range):
    num_assets = len(returns)
    weights = cp.Variable(num_assets)
    risk = cp.quad_form(weights, covariance)  # Portfolio risk (variance)
    expected_return = returns @ weights  # Portfolio expected return

    results = []
    for epsilon in epsilon_range:
        # Objective: Minimize risk
        objective = cp.Minimize(risk)
        
        # Constraints: adjust ε to explore the trade-off
        constraints = [
            cp.sum(weights) == 1,  # Weights sum to 1
            weights >= 0,  # No short selling
            expected_return >= epsilon  # Return must be at least ε
        ]
        
        # Solve the problem
        problem = cp.Problem(objective, constraints)
        problem.solve()
        
        if problem.status == cp.OPTIMAL:
            results.append({
                'epsilon': epsilon,
                'risk': risk.value,
                'return': expected_return.value,
                'weights': weights.value
            })

    return results


In [38]:
epsilon_range = np.linspace(returns.min(), returns.max(), 10)
portfolio_solutions = augmented_epsilon_constraint(returns, covariance_matrix, epsilon_range)

# Print the results
for solution in portfolio_solutions:
    print(f"Epsilon: {solution['epsilon']:.4f}, Risk: {solution['risk']:.4f}, Return: {solution['return']:.4f}, Weights: {solution['weights']}")


Epsilon: -0.0014, Risk: 0.0000, Return: 0.0020, Weights: [ 3.38169435e-01  5.08614682e-01  1.53215883e-01 -1.19246341e-18]
Epsilon: 0.0005, Risk: 0.0000, Return: 0.0020, Weights: [ 3.38169435e-01  5.08614682e-01  1.53215883e-01 -1.19246341e-18]
Epsilon: 0.0023, Risk: 0.0000, Return: 0.0023, Weights: [0.35458823 0.47138227 0.1657566  0.0082729 ]
Epsilon: 0.0042, Risk: 0.0000, Return: 0.0042, Weights: [0.39000542 0.34017985 0.16236176 0.10745297]
Epsilon: 0.0060, Risk: 0.0000, Return: 0.0060, Weights: [0.42542261 0.20897743 0.15896691 0.20663304]
Epsilon: 0.0078, Risk: 0.0000, Return: 0.0078, Weights: [0.4608398  0.07777501 0.15557207 0.30581312]
Epsilon: 0.0097, Risk: 0.0000, Return: 0.0097, Weights: [4.47144026e-01 1.24571400e-20 1.08362055e-01 4.44493919e-01]
Epsilon: 0.0115, Risk: 0.0001, Return: 0.0115, Weights: [3.59710446e-01 9.95396782e-25 1.05041432e-24 6.40289554e-01]
Epsilon: 0.0134, Risk: 0.0002, Return: 0.0134, Weights: [ 1.79855223e-01 -4.70578359e-24 -1.10439952e-24  8.201